In [66]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv

import constants

import os

from web3 import Web3
from bit import wif_to_key

In [48]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))
w3.isConnected()

True

In [49]:
# Load and set environment variables
load_dotenv("./mnenomic.dev")
mnemonic=os.getenv("MNEMONIC")
mnemonic

'best scorpion lucky orphan shoot holiday gesture color crazy field frown cattle'

In [50]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [52]:
coins = {}

coins[constants.ETH] = derive_wallets(mnemonic, constants.ETH, 3)
coins[constants.BTCTEST] = derive_wallets(mnemonic, constants.BTCTEST, 3)
coins[constants.BTC] = derive_wallets(mnemonic, constants.BTCTEST, 3)

print(coins[constants.ETH][0]['privkey'])
print(coins[constants.BTCTEST][0]['privkey'])
#print(coins[constants.BTC][0]['privkey'])

0xe4718458d4d6a0486c3b360837ba0a52463dbdff634bcb844e52c99e89adc5bc
cPkKfTgeKV1AKP7P1sWj3YDPw8dCbav3RcZjFgY8Q334SVJqFtBN


In [64]:
def priv_key_to_account(coin, priv_key):
    if coin == constants.ETH:
        return w3.eth.account.privateKeyToAccount(priv_key)
    if coin == constants.BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [84]:
eth_acc = priv_key_to_account(constants.ETH, coins[constants.ETH][0]['privkey'])
btctest_acc = priv_key_to_account(constants.BTCTEST, coins[constants.BTCTEST][0]['privkey'])

In [77]:
def create_trx(coin, account, to, amount):
    global trx_data
    if coin == constants.ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        trx_data = {
            "to": to,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == constants.BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [86]:
def send_trx(coin, account, to, amount):
    if coin == constants.ETH: 
        trx_eth = create_trx(coin,account, to, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest = create_trx(coin,account,to,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [91]:
raw_tx = send_trx(constants.ETH, eth_acc,"0xf9CF39bA70AD79fEDeC5E2303d65B997e456BEfC", w3.toWei(1, 'ether'))
raw_tx

0x46dbf3248884c6053e252471f0be991923ae8d6f09a6e25f841a0aa79d007988


HexBytes('0x46dbf3248884c6053e252471f0be991923ae8d6f09a6e25f841a0aa79d007988')